In [1]:

# ==================================================================================
#       Copyright (c) 2020 China Mobile Technology (USA) Inc. Intellectual Property.
#
#   Licensed under the Apache License, Version 2.0 (the "License");
#   you may not use this file except in compliance with the License.
#   You may obtain a copy of the License at
#
#          http://www.apache.org/licenses/LICENSE-2.0
#
#   Unless required by applicable law or agreed to in writing, software
#   distributed under the License is distributed on an "AS IS" BASIS,
#   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#   See the License for the specific language governing permissions and
#   limitations under the License.
# ==================================================================================
"""
lp entrypoint module

RMR Messages
 #define TS_UE_LIST 30000
for now re-use the 30000 to receive a UEID for prediction
"""

import os
import sys
sys.path.append('C:/Users/Mohammadreza/Desktop/My Class/Proj-DC/My Works/Scheduling/xApp/mr7-main')
import schedule
from zipfile import ZipFile
import json
from os import getenv
#from ricxappframe.xapp_frame import RMRXapp, rmr, Xapp
from mr import sdl
#from lp.exceptions import UENotFound, CellNotFound


import logging
import numpy as np
import torch
import tensorflow as tf
from numpy import zeros, newaxis
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from mr.db import DATABASE, DUMMY
import mr.populate as populate

///////////enter sdl.py//////////
UE_KEY_LIST= {'UEPDCPBytesDL', 'UEPDCPBytesUL', 'MeasPeriodUEPRBUsage', 'UEPRBUsageDL', 'MeasPeriodUEPDCPBytes', 'MeasTimestampUEPDCPBytes', 'UEPRBUsageUL', 'ServingCellID', 'MeasTimestampUEPRBUsage'}
CELL_KEY_LIST= {'MeasPeriodPDCPBytes', 'PDCPBytesUL', 'MeasTimestampAvailPRB', 'CellID', 'PDCPBytesDL', 'MeasPeriodAvailPRB', 'AvailPRBUL', 'MeasTimestampPDCPBytes', 'AvailPRBDL'}


C:\Users\Mohammadreza\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Mohammadreza\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Mohammadreza\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


///////////enter class DBCreatDrop/////////////
///////////enter class DATABASE(object)////////////////////
////////enter class Error in db////////////////
////////enter class NoDataError in db//////////////////
///////////////enter INSERTDATA class in populate/////////////


In [2]:
from tensorflow.keras.models import Sequential  # To compose multiple Layers
from tensorflow.keras.layers import LSTM, Dense  # Fully-Connected layer
from tensorflow.keras.layers import Activation  # Activation functions
from tensorflow.keras.optimizers import Adam

In [3]:
input_dim = 9

def LSTMClassifier():

    model = Sequential()

    model.add(LSTM(9, return_sequences=True, activation='relu', recurrent_activation='sigmoid', input_shape=(9,1)))

    model.add(LSTM(16, return_sequences=True, activation='relu', recurrent_activation='sigmoid'))

    model.add(LSTM(32, return_sequences=False, activation='relu', recurrent_activation='sigmoid'))

    model.add(Dense(32))
    model.add(Activation('relu'))

    # model.add(Dense(num_actions))
    # model.add(Activation('relu'))


    # model.add(Dense(num_actions))
    model.add(Dense(2))
    model.add(Activation('linear'))

    model.compile(loss='mse', optimizer=Adam(learning_rate=0.0001))
    print(model.summary())
    return model

In [4]:


xapp = None
print('first line of code: xapp=None=', xapp)
pos = 0
cell_data = None
rmr_xapp = None
ai_model = None

class UENotFound(BaseException):
    pass
class CellNotFound(BaseException):
    pass

def post_init(self):
    print('///////enter def post_init__/////////////////')
    """
    Function that runs when xapp initialization is complete
    """
    self.def_hand_called = 0
    self.traffic_steering_requests = 0


def handle_config_change(self, config):
    print('////////enter def handle_config_change//////////////')
    """
    Function that runs at start and on every configuration file change.
    """
    self.logger.debug("handle_config_change: config: {}".format(config))


def default_handler(self, summary, sbuf):
    print('/////////enter def default_handler///////////////')
    """
    Function that processes messages for which no handler is defined
    """
    self.def_hand_called += 1
    print('self.def_hand_called += 1=', self.def_hand_called)
    self.logger.warning("default_handler unexpected message type {}".format(summary[rmr.RMR_MS_MSG_TYPE]))
    self.rmr_free(sbuf)


def mr_req_handler(self, summary, sbuf):
    print('///////////enter def mr_req handler/////////////')
    """
    This is the main handler for this xapp, which handles load prediction requests.
    This app fetches a set of data from SDL, and calls the predict method to perform
    prediction based on the data

    The incoming message that this function handles looks like:
        {"UEPredictionSet" : ["UEId1","UEId2","UEId3"]}
    """
    self.traffic_steering_requests += 1
    # we don't use rts here; free the buffer
    self.rmr_free(sbuf)

    ue_list = []
    try:
        print('////enter first try in mr_req_handler////')
        print('rmr.RMR_MS_PAYLOAD=', rmr.RMR_MS_PAYLOAD)
        print('summary[rmr.RMR_MS_PAYLOAD]=', summary[rmr.RMR_MS_PAYLOAD])
        req = json.loads(summary[rmr.RMR_MS_PAYLOAD])  # input should be a json encoded as bytes
        print('req = json.loads(summary[rmr.RMR_MS_PAYLOAD])=', req)
        ue_list = req["UEPredictionSet"]
        print('ue_list=req["UEPredictionSet"] =', ue_list)
        self.logger.debug("mr_req_handler processing request for UE list {}".format(ue_list))
    except (json.decoder.JSONDecodeError, KeyError):
        print('////enter first except in mr_req_handler////')
        self.logger.warning("mr_req_handler failed to parse request: {}".format(summary[rmr.RMR_MS_PAYLOAD]))
        return
    print('ue_list mr_req_handler aftr 1st try=', ue_list)
    # iterate over the UEs, fetches data for each UE and perform prediction
    for ueid in ue_list:
        try:
            print('////enter second try in mr_req_handler////')
            uedata = sdl.get_uedata(self, ueid)
            print('uedata = sdl.get_uedata(self, ueid)=', uedata)
            predict(self, uedata)
            print('predict(self, uedata)=', predict(self, uedata))
        except UENotFound:
            print('////enter second except in mr_req_handler////')
            print('enter UENotFound in mr_req_handler')
            self.logger.warning("mr_req_handler received a TS Request for a UE that does not exist!")

def entry():
    print('////////////enter def entry///////////////')
    """  Read from DB in an infinite loop and run prediction every second
      TODO: do training as needed in the future
    """
    schedule.every(1).seconds.do(run_prediction)
    print('/////////pass 1 entry schedule.every(1).seconds.do(run_prediction, self)/////')
    while True:
        #print('////while True in entry/////') 
        schedule.run_pending()

def run_prediction():
    print('///////////////enter def run_prediction///////////////')
    """Read the latest cell_meas sample from influxDB and run it by the model inference
    """

    global pos
    sample = [3735, 0, 27648, 2295, 18, -1, 16383,-1, -1, -1]
    print('sample=[3735,...]=', sample)
    if cell_data:
        print('//////////enter if cell_data in run_prediction/////')
        print('cell_data=', cell_data)
        pos = (pos + 1) % len(cell_data)  # iterate through entire list one at a time
        print('pos=(pos + 1) % len(cell_data)=', pos)
        sample = cell_data[pos]
        print('sample = cell_data[pos]=', sample)
    predict(sample)

def predict(celldata):
    print('/////////////enter def predict/////////////////')      
    """
    This is the method that's to perform prediction based on a model
    For now it just returns dummy data
    :return:
    """
    ai_model = load_model_parameter()
    print('ai_model.summary()=', ai_model)
    print('celldata:', celldata)
    ret = predict_unseen_data(ai_model, celldata)
    print('ret=predict_unseen_data(ai_model, celldata)=', ret)
    print('celldata:', celldata)
    print('Classification', ret)
    print('////prediction doneee///////')
    return ret

def load_model_parameter():
    print('/////////////enter def load_model_parameters////////////////')
    PATH = 'model.pth'
    print('PATH = model.pth=', PATH)      
    cwd = os.getcwd()
    print('cwd=os.getcwd=', cwd)
    print('os.listdir(cwd)=', os.listdir(cwd))
    if not os.path.exists(PATH):
        print('///enter if not os.path.exists(PATH):////')
        with ZipFile('C:/Users/Mohammadreza/Desktop/My Class/Proj-DC/My Works/Scheduling/xApp/mr7-main/mr/model.zip', 'r') as zip:
            zip.printdir()
            zip.extractall()
    #input_dim = 10
    input_dim = 9
    hidden_dim = 256
    layer_dim = 3
    output_dim = 2
    device = "cpu"
    #model = LSTMClassifier(input_dim, hidden_dim, layer_dim, output_dim)
    #print('model=LSTClassifier=', model)
    
    model = LSTMClassifier()
    print('model=LSTClassifier=', model.summary())
    
    model.save(cwd)
    
    model = tf.keras.models.load_model(cwd)
    
#     print("Model's state_dict:")
#     for param_tensor in model.state_dict():
#         print(param_tensor, "\t", model.state_dict()[param_tensor].size())
#     # model = model.to(device)
#     torch.save(model.state_dict(), PATH)
    
#     model.load_state_dict(torch.load(PATH))
#     model.eval()
#     print('model.eval()=', model)      
    # print(model)
    return model

def predict_unseen_data(model, unseen_data):
    print('/////////////////enter def predict_unseen_data///////////////////')
    print('unseen_data=', unseen_data)
    #print('unseen_data[measTimeStampRf]=', unseen_data['measTimeStampRf'])
    np_data = np.asarray(unseen_data, dtype=np.float32)
    print('np_data=', np_data)
    print('newaxis=', newaxis)
    X_grouped = np_data[newaxis, newaxis, :]
    print('X_grouped = np_data[newaxis, newaxis, :]=',  X_grouped)
    print('X_grouped.transpose(0, 2, 1)=', X_grouped.transpose(0, 2, 1))
    
#     print('torch.tensor(X_grouped.transpose(0, 2, 1))=', torch.tensor(X_grouped.transpose(0, 2, 1)))
#     X_grouped = torch.tensor(X_grouped.transpose(0, 2, 1)).float()
#     print('X_grouped=', X_grouped)
#     y_fake = torch.tensor([0] * len(X_grouped)).long()
#     print('y_fake=', y_fake)
    
    print('X_grouped.transpose=', X_grouped.transpose)
    X_grouped = X_grouped.transpose(0,2,1)
    print('X_grouped.transpose(0,2,1)=', X_grouped)
    X_grouped = tf.convert_to_tensor(X_grouped, dtype=tf.float32)
    print('X_grouped=', X_grouped)
    y_fake = tf.convert_to_tensor([0] * len(X_grouped), dtype=tf.int64)
    print('y_fake=', y_fake)
    
#     tensor_test = TensorDataset(X_grouped, y_fake)
#     print('tensor_test= TensorDataset(X_grouped, y_fake)=', tensor_test)
    
    tensor_test = tf.data.Dataset.from_tensors((X_grouped, y_fake))
    print('tensor_test= TensorDataset(X_grouped, y_fake)=', tensor_test)
    
#     test_dl = DataLoader(tensor_test, batch_size=1, shuffle=False)
#     print('test_dl=', test_dl)    
    
    test_dl = tensor_test.batch(1)
    print('test_dl=', test_dl) 
    
    model.fit(tensor_test, epochs=20, verbose=1)
    
    ret = []
    for batch, _ in test_dl:
        print('///enter for loop in predict unseen data////')
        #batch = batch.permute(0, 2, 1)
        print('batch=', batch)
        batch = batch[0]
        print('batch = batch[0]=', batch)
        #batch = tf.transpose(batch, perm=[0,2,1])
        #batch = tf.transpose(batch)
        #print('batch = tf.transpose(batch, (0,2,1))=', batch)
        
        #batch = batch.transpose(0,2,1)
        #print('batch = batch.transpose(0,2,1)=', batch)
        #batch = tf.keras.layers.Permute((0,2,1), batch)
        #print('batch= batch.permute(0, 2, 1)=', batch)
        
        out = model.predict(batch)
        print('out=model(batch)=', out)
        
        y_hat = tf.nn.log_softmax(out)
        print('y_hat = tf.nn.log_softmax(out)=', y_hat)
        y_hat = tf.argmax(y_hat)
        print('y_hat = tf.argmax(y_hat)=', y_hat)
        #y_hat = F.log_softmax(out, dim=1).argmax(dim=1)
        #print('y_hat= F.log_softmax(out, dim=1).argmax(dim=1)=', y_hat)
        
        y_hat = tf.get_static_value(y_hat, partial=False)
        print('y_hat = tf.get_static_value(y_hat, partial=False)=', y_hat)
        
        ret += y_hat.tolist()
        print('ret += y_hat.tolist()=', ret)
    print('ret after for loop=', ret)
    print('ret[0]', ret[0]) 
    if ret[0] == 0:
        print('ret[0] == 0')  
        return "Normal"
    return "Congestion"

def connectdb(thread=False):
    print('////////////////////enter def connectdb///////////////////')
    # Create a connection to InfluxDB if thread=True, otherwise it will create a dummy data instance
    global db
    global cell_data
    if thread:
        print('///////////////enter if(thread) in connectdb///////////////////')  
        db = DUMMY()
        print('db =DUMMY()=', db)  
    else:
        print('//////enter else= populate.populate()////////////////')  
        populate.populatedb()  # temporary method to populate db, it will be removed when data will be coming through KPIMON to influxDB
        
        print('////came back from populate to connectdb.else:, db=DATABASE(CellData)///////')
        db = DATABASE('CellData')
        print('////came back from db.DATABASE-init to connectdb.else///////')
        print('db =  DATABASE(celldata) =', db) 
        db.read_data("cellMeas")
        print('////came back from db.DATABASE-read-data to connectdb.else///////')
        print('db.read_data("cellMeas")=', db.read_data("cellMeas"))
        cell_data = db.data.values.tolist()  # needs to be updated in future when live feed will be coming through KPIMON to influxDB
        print('cell_data = db.data.values.tolist()=', cell_data)
        #print('cell_data:, cell_data)
        print('///////connectdb finished go to start//////')

def start(thread=False):
 
    print('////////////////entered Starrrrrrrrrrrt///////////////////')
    """
    This is a convenience function that allows this xapp to run in Docker
    for "real" (no thread, real SDL), but also easily modified for unit testing
    (e.g., use_fake_sdl). The defaults for this function are for the Dockerized xapp.
    """
    global xapp, ai_model
    #fake_sdl = getenv("USE_FAKE_SDL", None)
    #xapp = Xapp(entrypoint=entry, rmr_port=4560, use_fake_sdl=False)
    
    connectdb(thread)
    print('///////come back from connectdb////////')
    ai_model = load_model_parameter()
    print('ai_model.summary in start=', ai_model)
    
    use_fake_sdl=False
    rmr_port=4560
    #xapp= entry(self)
    #xapp.run()
    entry()
    
    #fake_sdl = getenv("USE_FAKE_SDL", None)
    #xapp = Xapp(entrypoint=entry, rmr_port=4560, use_fake_sdl=False)
    #print('xapp = Xapp(entrypoint=entry, rmr_port=4560, use_fake_sdl=fake_sdl)=', xapp)
    #connectdb(thread)
    #ai_model = load_model_parameter()
    #print('ai_model.summary=', ai_model)
    
    #xapp.run()


def stop():
    print('/////////////enter def stop//////////////////')      
    """
    can only be called if thread=True when started
    """
    xapp.stop()


def get_stats():
    print('//////////////////enter def get_stats()////////////////////')
    """
    hacky for now, will evolve
    """
    print('DefCalled:rmr_xapp.def_hand_called=', rmr_xapp.def_hand_called)
    print('SteeringRequests:rmr_xapp.traffic_steering_requests=', rmr_xapp.traffic_steering_requests) 
    return {"DefCalled": rmr_xapp.def_hand_called,
            "SteeringRequests": rmr_xapp.traffic_steering_requests}

# class LSTMClassifier(nn.Module):
#     print('/////////////////enter class LSTClassifier////////////////////')      
#     def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
#         print('///////enter def __init__ in LST//////////////////')  
#         super().__init__()
#         self.hidden_dim = hidden_dim
#         print('self.hidden_dim=', self.hidden_dim)
#         self.layer_dim = layer_dim
#         print('self.layer_dim=', self.layer_dim)
#         self.rnn = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, output_dim)
#         self.batch_size = None
#         self.hidden = None

#     def forward(self, x):
#         print('////////enter def forward in LST//////////////////') 
#         h0, c0 = self.init_hidden(x)
#         print('h0, c0 = self.init_hidden(x)', h0)
#         print('h0, c0 = self.init_hidden(x)', c0)
#         out, (hn, cn) = self.rnn(x, (h0, c0))
#         out = self.fc(out[:, -1, :])
#         print('out = self.fc(out[:, -1, :])=', out)
#         return out

#     def init_hidden(self, x):
#         print('/////////enter def init_hidden in LST//////////////////')   
#         h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
#         print('h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)', h0)
#         c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
#         print('c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)', c0)
#         return [t for t in (h0, c0)]


first line of code: xapp=None= None


In [5]:
print('////////////////entered Starrrrrrrrrrrt///////////////////')
"""
This is a convenience function that allows this xapp to run in Docker
for "real" (no thread, real SDL), but also easily modified for unit testing
(e.g., use_fake_sdl). The defaults for this function are for the Dockerized xapp.
"""
thread = False
global xapp, ai_model
#fake_sdl = getenv("USE_FAKE_SDL", None)
#xapp = Xapp(entrypoint=entry, rmr_port=4560, use_fake_sdl=False)

connectdb(thread)
print('///////come back from connectdb////////')
ai_model = load_model_parameter()
print('ai_model.summary in start=', ai_model)

use_fake_sdl=False
rmr_port=4560
#xapp= entry(self)
#xapp.run()
entry()

////////////////entered Starrrrrrrrrrrt///////////////////
////////////////////enter def connectdb///////////////////
//////enter else= populate.populate()////////////////
/////////////enter def populatedb()///////////
data = pd.read_csv(mr/cells.csv)=    du-id          measTimeStampRf  availPrbDl  availPrbUl  measPeriodPrb  \
0   1001  2020-09-16T13:57:48.220         273         273             10   

   nrCellIdentity  pdcpBytesUl  pdcpBytesDl  measPeriodPdcpBytes  
0               2      5000000      5000000                   10  
///////////////enter def time//////////////
df.index= DatetimeIndex(['2022-08-13 18:33:59.684690'], dtype='datetime64[ns]', freq='10L')
2022-08-13 18:33:59.684690    2020-09-16T13:57:48.220
Freq: 10L, Name: measTimeStampRf, dtype: object
lambda x: str(x)= <function time.<locals>.<lambda> at 0x000001EAF678C280>
df=                             du-id          measTimeStampRf  availPrbDl  \
2022-08-13 18:33:59.684690   1001  2020-09-16T13:57:48.220         273

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 9, 9)              396       
_________________________________________________________________
lstm_1 (LSTM)                (None, 9, 16)             1664      
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                6272      
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
activation (Activation)      (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 66        
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0

1/1 [==============================] - 2s 2s/step - loss: 17723953371119922502959104.0000
Epoch 2/20
1/1 [==============================] - 0s 13ms/step - loss: nan
Epoch 3/20
1/1 [==============================] - 0s 13ms/step - loss: nan
Epoch 4/20
1/1 [==============================] - 0s 9ms/step - loss: nan
Epoch 5/20
1/1 [==============================] - 0s 9ms/step - loss: nan
Epoch 6/20
1/1 [==============================] - 0s 9ms/step - loss: nan
Epoch 7/20
1/1 [==============================] - 0s 9ms/step - loss: nan
Epoch 8/20
1/1 [==============================] - 0s 11ms/step - loss: nan
Epoch 9/20
1/1 [==============================] - 0s 7ms/step - loss: nan
Epoch 10/20
1/1 [==============================] - 0s 7ms/step - loss: nan
Epoch 11/20
1/1 [==============================] - 0s 7ms/step - loss: nan
Epoch 12/20
1/1 [==============================] - 0s 8ms/step - loss: nan
Epoch 13/20
1/1 [==============================] - 0s 8ms/step - loss: nan
Epoch 14/20
1/1

1/1 [==============================] - 0s 12ms/step - loss: nan
Epoch 7/20
1/1 [==============================] - 0s 10ms/step - loss: nan
Epoch 8/20
1/1 [==============================] - 0s 9ms/step - loss: nan
Epoch 9/20
1/1 [==============================] - 0s 10ms/step - loss: nan
Epoch 10/20
1/1 [==============================] - 0s 12ms/step - loss: nan
Epoch 11/20
1/1 [==============================] - 0s 10ms/step - loss: nan
Epoch 12/20
1/1 [==============================] - 0s 11ms/step - loss: nan
Epoch 13/20
1/1 [==============================] - 0s 11ms/step - loss: nan
Epoch 14/20
1/1 [==============================] - 0s 10ms/step - loss: nan
Epoch 15/20
1/1 [==============================] - 0s 9ms/step - loss: nan
Epoch 16/20
1/1 [==============================] - 0s 8ms/step - loss: nan
Epoch 17/20
1/1 [==============================] - 0s 7ms/step - loss: nan
Epoch 18/20
1/1 [==============================] - 0s 7ms/step - loss: nan
Epoch 19/20
1/1 [===============

1/1 [==============================] - 0s 10ms/step - loss: 4.7007e-09
Epoch 12/20
1/1 [==============================] - 0s 10ms/step - loss: 1.2604e-09
Epoch 13/20
1/1 [==============================] - 0s 9ms/step - loss: 1.1937e-08
Epoch 14/20
1/1 [==============================] - 0s 9ms/step - loss: 2.2092e-08
Epoch 15/20
1/1 [==============================] - 0s 9ms/step - loss: 2.0615e-08
Epoch 16/20
1/1 [==============================] - 0s 9ms/step - loss: 1.0233e-08
Epoch 17/20
1/1 [==============================] - 0s 8ms/step - loss: 1.3506e-09
Epoch 18/20
1/1 [==============================] - 0s 7ms/step - loss: 1.0430e-09
Epoch 19/20
1/1 [==============================] - 0s 8ms/step - loss: 7.2343e-09
Epoch 20/20
1/1 [==============================] - 0s 7ms/step - loss: 1.2096e-08
///enter for loop in predict unseen data////
batch= tf.Tensor(
[[[[2.7300000e+02]
   [2.7300000e+02]
   [1.0010000e+03]
   [1.0000000e+01]
   [1.0000000e+01]
   [1.6604157e+18]
   [2.0000000

...

1/1 [==============================] - 0s 13ms/step - loss: nan
Epoch 11/20
1/1 [==============================] - 0s 11ms/step - loss: nan
Epoch 12/20
1/1 [==============================] - 0s 12ms/step - loss: nan
Epoch 13/20
1/1 [==============================] - 0s 11ms/step - loss: nan
Epoch 14/20
1/1 [==============================] - 0s 10ms/step - loss: nan
Epoch 15/20
1/1 [==============================] - 0s 11ms/step - loss: nan
Epoch 16/20
1/1 [==============================] - 0s 10ms/step - loss: nan
Epoch 17/20
1/1 [==============================] - 0s 7ms/step - loss: nan
Epoch 18/20
1/1 [==============================] - 0s 10ms/step - loss: nan
Epoch 19/20
1/1 [==============================] - 0s 8ms/step - loss: nan
Epoch 20/20
1/1 [==============================] - 0s 11ms/step - loss: nan
///enter for loop in predict unseen data////
batch= tf.Tensor(
[[[[2.7300000e+02]
   [2.7300000e+02]
   [1.0010000e+03]
   [1.0000000e+01]
   [1.0000000e+01]
   [1.6604157e+18]
 

1/1 [==============================] - 0s 8ms/step - loss: nan
Epoch 16/20
1/1 [==============================] - 0s 8ms/step - loss: nan
Epoch 17/20
1/1 [==============================] - 0s 8ms/step - loss: nan
Epoch 18/20
1/1 [==============================] - 0s 8ms/step - loss: nan
Epoch 19/20
1/1 [==============================] - 0s 9ms/step - loss: nan
Epoch 20/20
1/1 [==============================] - 0s 9ms/step - loss: nan
///enter for loop in predict unseen data////
batch= tf.Tensor(
[[[[2.7300000e+02]
   [2.7300000e+02]
   [1.0010000e+03]
   [1.0000000e+01]
   [1.0000000e+01]
   [1.6604157e+18]
   [2.0000000e+00]
   [5.0000000e+06]
   [5.0000000e+06]]]], shape=(1, 1, 9, 1), dtype=float32)
batch = batch[0]= tf.Tensor(
[[[2.7300000e+02]
  [2.7300000e+02]
  [1.0010000e+03]
  [1.0000000e+01]
  [1.0000000e+01]
  [1.6604157e+18]
  [2.0000000e+00]
  [5.0000000e+06]
  [5.0000000e+06]]], shape=(1, 9, 1), dtype=float32)
out=model(batch)= [[nan nan]]
y_hat = tf.nn.log_softmax(out)= t

1/1 [==============================] - 0s 8ms/step - loss: nan
///enter for loop in predict unseen data////
batch= tf.Tensor(
[[[[2.7300000e+02]
   [2.7300000e+02]
   [1.0010000e+03]
   [1.0000000e+01]
   [1.0000000e+01]
   [1.6604157e+18]
   [2.0000000e+00]
   [5.0000000e+06]
   [5.0000000e+06]]]], shape=(1, 1, 9, 1), dtype=float32)
batch = batch[0]= tf.Tensor(
[[[2.7300000e+02]
  [2.7300000e+02]
  [1.0010000e+03]
  [1.0000000e+01]
  [1.0000000e+01]
  [1.6604157e+18]
  [2.0000000e+00]
  [5.0000000e+06]
  [5.0000000e+06]]], shape=(1, 9, 1), dtype=float32)
out=model(batch)= [[nan nan]]
y_hat = tf.nn.log_softmax(out)= tf.Tensor([[nan nan]], shape=(1, 2), dtype=float32)
y_hat = tf.argmax(y_hat)= tf.Tensor([0 0], shape=(2,), dtype=int64)
y_hat = tf.get_static_value(y_hat, partial=False)= [0 0]
ret += y_hat.tolist()= [0, 0]
ret after for loop= [0, 0]
ret[0] 0
ret[0] == 0
ret=predict_unseen_data(ai_model, celldata)= Normal
celldata: [273, 273, 1001, 10, 10, 1660415639684690000, 2, 5000000, 5

out=model(batch)= [[nan nan]]
y_hat = tf.nn.log_softmax(out)= tf.Tensor([[nan nan]], shape=(1, 2), dtype=float32)
y_hat = tf.argmax(y_hat)= tf.Tensor([0 0], shape=(2,), dtype=int64)
y_hat = tf.get_static_value(y_hat, partial=False)= [0 0]
ret += y_hat.tolist()= [0, 0]
ret after for loop= [0, 0]
ret[0] 0
ret[0] == 0
ret=predict_unseen_data(ai_model, celldata)= Normal
celldata: [273, 273, 1001, 10, 10, 1660415639684690000, 2, 5000000, 5000000]
Classification Normal
////prediction doneee///////
///////////////enter def run_prediction///////////////
sample=[3735,...]= [3735, 0, 27648, 2295, 18, -1, 16383, -1, -1, -1]
//////////enter if cell_data in run_prediction/////
cell_data= [[273, 273, 1001, 10, 10, 1660415639684690000, 2, 5000000, 5000000]]
pos=(pos + 1) % len(cell_data)= 0
sample = cell_data[pos]= [273, 273, 1001, 10, 10, 1660415639684690000, 2, 5000000, 5000000]
/////////////enter def predict/////////////////
/////////////enter def load_model_parameters////////////////
PATH = model

KeyboardInterrupt: 